In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !unzip '/content/drive/MyDrive/Programming(II)/Lesson13_AOIDemo/aoi.zip'
drive_path = '/content/drive/MyDrive/aoi/'

In [ ]:
!unzip '/content/drive/MyDrive/aoi/train_images.zip'

In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import os

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
      # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
from pandas import read_csv
import pandas as pd
# load file as CSV
from sklearn.preprocessing import LabelBinarizer
import numpy
import numpy as np
filename = '/content/drive/MyDrive/aoi/train.csv'
mapping1_csv = read_csv(filename,encoding="Big5")
filename = '/content/drive/MyDrive/aoi/test.csv'
mapping2_csv = read_csv(filename,encoding="Big5")
mapping1_csv = mapping1_csv.values.tolist()
mapping2_csv = mapping2_csv.values.tolist()

In [ ]:
print(type(mapping1_csv))
print(len(mapping1_csv))
print(mapping1_csv[0])
print(mapping2_csv[0])

In [ ]:
import cv2
import os
from PIL import Image
from numpy import savez_compressed
from numpy import asarray

def load_dataset(directory,mapping_csv):
    X, y = list(), list()
    data = []
    labels = []
    i=0
    # loop over the image paths
    for imagePath, label in mapping_csv:
        image = Image.open(directory + imagePath)
        image = image.convert('RGB')
        image = image.resize((224,224))
        image = asarray(image)
        data.append(image)
        labels.append(label)
    # convert the data and labels to NumPy arrays while scaling the pixel
    # perform one-hot encoding on the labels
    from keras import utils as np_utils
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    X.extend(data)
    y.extend(labels)
    return asarray(X), asarray(y)

In [ ]:
trainX, trainy = load_dataset('train_images/',mapping1_csv)
print(trainX.shape, trainy.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout, Flatten, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.layers import Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19

In [ ]:
# input_tensor = Input(shape=(224, 224, 3))
# model = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
# model_1 = Sequential()
# for layer in model.layers:
#   print(layer)
#   model_1.add(layer)
# model_1.add(Flatten())
# model_1.add(Dense(1000, activation = "softmax"))
# model_1.add(Dense(6, activation = "softmax"))
# model_1.summary()


In [ ]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Dropout(0.2)(top_model)
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dropout(0.2)(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
model=create_model((224,224,3),6,optimizer='rmsprop',fine_tune=1)

In [ ]:
model.summary()

In [ ]:
model.fit(trainX, trainy , batch_size=20, epochs=30)

In [ ]:
model.evaluate(trainX,trainy)

In [ ]:
model.save('model_VGG16_fine_tune_5.h5')

In [ ]:
!unzip '/content/drive/MyDrive/aoi/test_images.zip'

In [ ]:
def load_dataset(directory,mapping_csv):
    X= list()
    data = []
    i=0
    # loop over the image paths
    for imagePath, label in mapping_csv:
        image = Image.open(directory + imagePath)
        image = image.convert('RGB')
        image = image.resize((224,224))
        image = asarray(image)
        data.append(image)
    # convert the data and labels to NumPy arrays while scaling the pixel
    # perform one-hot encoding on the labels
    from keras import utils as np_utils
    X.extend(data)
    return asarray(X)

In [ ]:
testX= load_dataset('test_images/',mapping2_csv)
print(testX.shape)

In [ ]:
y_pred=model.predict(testX)
y_pred

In [ ]:
test_result=[]
for pred in y_pred:
  test_result.append(np.argmax(pred))
test_result=np.array(test_result).reshape(-1,1)
test_result.shape

In [ ]:
df_test=pd.read_csv('/content/drive/MyDrive/aoi/test.csv')
df_test

In [ ]:
df_test.Label=test_result
df_test.to_csv('final_10.csv',index=None)